# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
A. Bayer  ->  M. A. Bayer  |  ['A. Bayer']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
J. Wolf  ->  J. Wolf  |  ['J. Wolf']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
A. Dumont  ->  A. Dumont  |  ['A. Dumont']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
S. Bosman  ->  S. Bosman  |  ['S. Bosman']
F. Walter  ->  F. Walter  |  ['F. Walter']
N. Storm  ->  N. Storm  |  ['N. Storm']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
G. Mattia  ->  G. Mattia  |  ['G. Mattia']


R. Keenan  ->  R. Keenan  |  ['R. Keenan']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
H. Linz  ->  H. Linz  |  ['H. Linz']
N. Wang  ->  N. Wang  |  ['N. Wang']
T. Henning  ->  T. Henning  |  ['T. Henning']


K. Naidoo  ->  K. Naidoo  |  ['K. Naidoo']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 124 new papers today
          15 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/15 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.15812


extracting tarball to tmp_2512.15812... done.


/tmp/ipykernel_2859/2822249172.py:52: LatexWarning: 2512.15812 did not run properly
bad escape \e at position 14
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2512.15833


...

extracting tarball to tmp_2512.16568...

 done.
Retrieving document from  https://arxiv.org/e-print/2512.16844


extracting tarball to tmp_2512.16844...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2512.16844/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'Sec4_Conclusion.tex' from 'tmp_2512.16844/Sec4_Conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'Sec3_Result_p2.tex' from 'tmp_2512.16844/Sec3_Result_p2.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15856-b31b1b.svg)](https://arxiv.org/abs/2512.15856) | **The AIDA-TNG project: 3D halo shapes**  |
|| C. Giocoli, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *12 pages, and 8 figures, submitted to A&A. Find more information on AIDA-TNG here: this https URL*|
|**Abstract**|            Context. The shapes of dark matter halos can be used to constrain the fundamental properties of dark matter. In standard Cold Dark Matter (CDM) cosmologies, halos are typically triaxial, with a preference for prolate configurations, particularly at low masses and high redshift. Aims. We focus on the characterization of total matter 3D shape in alternative dark matter models, such as Self-Interacting Dark Matter (SIDM) and Warm Dark Matter (WDM). These scenarios predict different structural properties due to collisional effects or the suppression of small-scale power. Methods. We measure the different halo component shapes - dark matter, stars and gas - at various radii from the center in the AIDA-TNG (Alternative Interacting Dark Matter and Astrophysics - TNG), which is a suite of high-resolution cosmological simulations built upon the IllustrisTNG framework. The intent is to systematically study how different dark matter models - specifically, SIDM and WDM - affect galaxy formation and the structure of dark matter halos, when realistic baryonic physics is also included. Results. SIDM models tend to produce rounder and more isotropic halos, especially in the inner regions, as a result of momentum exchange between dark matter particles. WDM halos are also slightly more spherical than their CDM counterparts, and are typically less concentrated. In all cases, the inclusion of self-consistent baryonic physics makes the central regions of all halos rounder, while still revealing clear distinctions among the various dark matter models. Conclusions. The general framework presented in this work, based on the 3D halo shape, can be useful to interpret multi-wavelength data analyses of galaxies and clusters.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15869-b31b1b.svg)](https://arxiv.org/abs/2512.15869) | **The AIDA-TNG project: dark matter profiles and concentrations in alternative dark matter models**  |
|| G. Despali, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *17 pages, 12 figures, submitted to A&A. Comments welcome. Find more information on AIDA-TNG here: this https URL*|
|**Abstract**|            In the standard Cold Dark Matter (CDM) scenario, the density profiles of dark matter haloes are well described by analytical models linking their concentration to halo mass. Alternative scenarios, such as warm dark matter (WDM) and self-interacting dark matter (SIDM), modify the inner structure of haloes and predict different profile shapes and central slopes. We employ the AIDA-TNG simulations to investigate how alternative dark matter physics and baryonic processes jointly shape the internal structure of haloes. Using dark-matter-only and full-physics runs, we measure the dark matter density profiles of haloes spanning six orders of magnitude in mass, from 10^9.5 Msun to 10^14.5 Msub, and characterise them with multiple analytical models. We provide the distribution of the best-fitting parameters, as well as the concentration-mass relation in WDM and SIDM. The Einasto profile well reproduces the inner flattening produced in WDM models, both in the collisionless and in the full-physics runs. In SIDM dark-matter-only runs, haloes are better described by explicitly cored profiles, with core sizes that depend on mass and on the self-interaction model. When baryons are included, the differences between CDM and SIDM decrease, and such large dark-matter cores no longer form because adiabatic contraction in the baryon-dominated region counteracts self-interactions. Nevertheless, the coupling between baryons and self-interactions induces a broader range of inner slopes, including cases that are steeper than CDM at Milky Way masses. Alternative dark matter physics thus leaves clear signatures in the inner halo structure, even if baryons significantly reshape these differences. Our results are useful for future studies that need to predict the properties of haloes in multiple dark matter models.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15881-b31b1b.svg)](https://arxiv.org/abs/2512.15881) | **Shedding the envelope: JWST reveals a kiloparsec-scale [OIII]-weak Balmer shell around a z=7.64 quasar**  |
|| <mark>J. Wolf</mark>, et al. -- incl., <mark>E. Bañados</mark>, <mark>A. Dumont</mark>, <mark>S. Belladitta</mark>, <mark>S. Bosman</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *Revised version submitted after the first referee review*|
|**Abstract**|            Luminous quasars at the redshift frontier z>7 serve as stringent probes of super-massive black hole formation and they are thought to undergo much of their growth obscured by dense gas and dust in their host galaxies. Fully characterizing the symbiotic evolution of SMBHs and hosts requires rest-frame optical observations that span spatial scales from the broad-line region to the ISM and CGM. JWST now provides the necessary spatially resolved spectroscopy to do so. But the physical conditions that regulate the interplay between SMBHs and their hosts at the highest redshifts, especially the nature of early feedback phases, remain unclear. We present JWST/NIRSpec IFU observations of J0313$-$1806 at z=7.64, the most distant luminous quasar known. From the restframe optical spectrum of the unresolved quasar, we derive a black hole mass of $M_\mathrm{BH}=(1.63 \pm 0.10)\times10^9 M_\odot$ based on H$\beta$ and an Eddington rate of $\lambda=L/L_\mathrm{Edd}=0.80\pm 0.05$, consistent with previous MgII-based estimates. J0313-1806 exhibits no detectable [O III] emission on nuclear scales. Most remarkably, we detect an ionized gas shell extending out to $\sim 1.8$ kpc traced by H$\beta$ emission that also lacks any significant [O III], with a $3\sigma$ upper limit on the [O III]$ \lambda$5007 to H$\beta$ flux ratio of $\log_{10} \left( F(\mathrm{[OIII]})/F(\mathrm{H}\beta)\right)=-1.15$. Through photoionization modelling, we demonstrate that the extended emission is consistent with a thin, clumpy outflowing shell where [OIII] is collisionally de-excited by dense gas. We interpret this structure as a fossil remnant of a recent blowout phase, providing evidence for episodic feedback cycles in one of the earliest quasars. These findings suggest that dense ISM phases may play a crucial role in shaping the spectral properties of quasars accross cosmic time.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15888-b31b1b.svg)](https://arxiv.org/abs/2512.15888) | **Observational constraints on the origin of the elements. X. Combining NLTE and machine learning for chemical diagnostics of 4 million stars in the 4MIDABLE-HR survey**  |
|| <mark>N. Storm</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>G. Guiglion</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *Submitted to ApJ. 12 pages + 22 pages appendix, 7 figures*|
|**Abstract**|            We present 4MOST-HR resolution Non-Local Thermal Equilibrium (NLTE) Payne artificial neural network (ANN), trained on 404,793 new FGK spectra with 16 elements computed in NLTE. This network will be part of the Stellar Abundances and atmospheric Parameters Pipeline (SAPP), which will analyse 4 million stars during the five year long 4MOST consortium 4: MIlky way Disc And BuLgE High-Resolution (4MIDABLE-HR) survey. A fitting algorithm using this ANN is also presented that is able to fully-automatically and self-consistently derive both stellar parameters and elemental abundances. The ANN is validated by fitting 121 observed spectra of low-mass FGKM type stars, including main-sequence dwarf, subgiant and giant stars down to [Fe/H] $\approx -3.4$ degraded to 4MOST-HR resolution, and comparing the derived abundances with the output of the classical radiative transfer code TSFitPy. We are able to recover all 18 elemental abundances with a bias <0.13 and spread <0.16 dex, although the typical values are <0.09 dex for most elements. These abundances are compared to the OMEGA+ Galactic Chemical Evolution model, showcasing for the first time, the expected performance and results obtained from high-resolution spectra of the quality expected to be obtained with 4MOST. The expected Galactic trends are recovered, and we highlight the potential of using many chemical elements to constrain the formation history of the Galaxy.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.16278-b31b1b.svg)](https://arxiv.org/abs/2512.16278) | **Radio frequency interference identification using eigenvalue decomposition for multi-beam observations**  |
|| J. Bai, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *9 pages, 7 figures, 2 tables. Accepted for publication in PASA*|
|**Abstract**|            With the installation of next-generation phased array feed (PAF) receivers on radio telescopes, there is an urgent need to develop effective and computationally efficient radio frequency interference (RFI) mitigation methods for large-scale surveys. Here we present a new RFI mitigation package, called mRAID (multi-beam RAdio frequency Interference Detector), which uses the eigenvalue decomposition algorithm to identify RFI in cross-correlation matrix (CCM) of data recorded by multiple beams. When applied to high time-resolution pulsar search data from the Five-hundred-meter Aperture Spherical Radio Telescope (FAST), mRAID demonstrates excellent performance in identifying RFI over short timescales, thereby enhancing the efficiency of pulsar and fast radio burst (FRB) searches. Since the computation of the CCM and the eigenvalue decomposition for each time sub-integration and frequency channel are independent, the process is fully parallelisable. As a result, mRAID offers a significant computational advantage over commonly used RFI detection methods.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.16568-b31b1b.svg)](https://arxiv.org/abs/2512.16568) | **A Tidal Disruption Event from an Intermediate-mass Black Hole Revealed by Comprehensive Multi-wavelength Observations**  |
|| J. Wang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *35 pages, 10 figures*|
|**Abstract**|            Tidal disruption events (TDEs) occur when a star crosses the tidal radius of a black hole (BH) and is ripped apart, providing a novel and powerful way to probe dormant BHs over a wide mass range. In this study, we present our late-time observations and comprehensive multi-wavelength analyses of an extraordinary TDE at the center of a dwarf galaxy, which exhibited successive flares in the optical, X-ray, and radio bands. Notably, we discovered an unexpected high-state X-ray plateau phase following the peak until the present time. Along with its reported prolonged rise lasting at least 550 days, these unique characteristics are consistent with the scenario of a TDE caused by an intermediate-mass black hole (IMBH) with a mass of approximately $(1-6) \times 10^5$ solar masses. Furthermore, scaling relations derived from the host-galaxy properties indicated a similar BH mass in concert. This discovery highlights the invaluable role of TDEs in the search for elusive IMBHs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.16844-b31b1b.svg)](https://arxiv.org/abs/2512.16844) | **Connecting current and future dual AGN searches to LISA and PTA gravitational wave detections**  |
|| N. Chen, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *20 pages, 16 figures; comments are welcome*|
|**Abstract**|            Dual active galactic nuclei (DAGN) mark an observable stage of massive black hole (MBH) pairing in galaxy mergers and are precursors to the MBH binaries that generate low-frequency gravitational waves. Using the large-volume ASTRID cosmological simulation, we construct DAGN catalogs matched to current (COSMOS-Web, DESI) and forthcoming (AXIS, Roman) searches. With realistic selection functions applied, ASTRID reproduces observed dual fractions, separations, and host-galaxy properties across redshifts. We predict a substantial population of small-separation (<5 kpc) duals that current surveys fail to capture, indicating that the apparent paucity of sub-kpc systems in COSMOS-Web is driven primarily by selection effects rather than a physical deficit. By following each simulated dual forward in time, we show that dual AGN are robust tracers of MBH mergers: ~30-70% coalesce within $\lesssim 1$ Gyr, and 20-60% of these mergers produce gravitational-wave signals detectable by LISA. Duals accessible to AXIS and Roman are the progenitors of ~10% of low-redshift LISA events and ~30% of the PTA-band stochastic background. Massive green-valley galaxies with moderate-luminosity AGN, together with massive star-forming hosts containing bright quasars at $z>1$, emerge as the most likely environments for imminent MBH binaries. These results provide a unified cosmological framework linking dual AGN demographics, MBH binary formation, and gravitational-wave emission, and they identify concrete, high-priority targets for coordinated electromagnetic and GW searches in upcoming multi-messenger surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.15812-b31b1b.svg)](https://arxiv.org/abs/2512.15812) | **Milky Way disc & Bulge in situ populations: ESO white paper - Expanding horizons call**  |
|| <mark>M. Bergemann</mark>, et al. |
|*Appeared on*| *2025-12-19*|
|*Comments*| *3 pages + cover. Submitted to ESO White Papers - Expanding Horizons call*|
|**Abstract**|            The formation and evolution of the Milky Way's disc, bar, and bulge remain fundamentally limited by the lack of a contiguous, Galaxy-wide, high-precision chemo-dynamical map. Key open questions - including the survival or destruction of the primitive discs, the origin of the bulge's multi-component structure, the role of mergers and secular processes, and the coupling between stellar chemistry, dynamics, and the Galactic potential - cannot be fully resolved with current or planned facilities. Existing spectroscopic surveys provide either high resolution for small samples or wide coverage at insufficient resolution and depth, and none can obtain homogeneous abundances, precise 3D kinematics, and reliable ages for the millions of stars required, particularly in the obscured midplane, the far side of the bar, or the outer, low-density disc. A new wide-field, massively multiplexed, large-aperture spectroscopic facility, capable of both high- and low-resolution spectroscopy over tens of thousands of square degrees, is therefore essential. Such a facility would deliver the statistical power, sensitivity, and completeness needed to reconstruct the Galaxy's assembly history, constrain its gravitational potential, and establish the Milky Way as the definitive benchmark for galaxy evolution.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 14</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.16361-b31b1b.svg)](https://arxiv.org/abs/2512.16361) | **Maximizing the sensitivity of ELT to habitable worlds with a space-based starshade**  |
|| M. Janson, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-12-19*|
|*Comments*| *White Paper - ESO 2040 Expanding Horizons, 3 pages + references, no figures*|
|**Abstract**|            The ELT will provide groundbreaking science across a wide range of areas, including small habitable-zone exoplanets; however, true Earth analogs in the habitable zones of Sun-like stars are generally beyond the reach even of the ELT, due to the extreme contrast ratio and small angular separation between the planet and star. Here, we note that the combination of ELT and a space-based starshade would provide the contrast needed to observe potentially tens of Earth analogs, as well as other planets. This would yield the scientific basis needed for addressing central scientific questions regarding the frequency and distribution of habitability and life in the Universe. The huge aperture of ELT, combined with a contrast otherwise only reachable in space, opens up scientific avenues that are unmatched by any other existing or foreseen facility. ESO could conceivably collaborate with ESA (and others) to facilitate a starshade mission suitable for synergy with the ELT, as well as to prepare the ELT instrumentation in order to maximize its potential for synergy with a starshade.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.15856/./figures/output_1_cut-0.png', 'tmp_2512.15856/./figures/output_1_cut-1.png', 'tmp_2512.15856/./figures/output_2_cut-0.png', 'tmp_2512.15856/./figures/output_2_cut-1.png', 'tmp_2512.15856/./figures/top_cut-0.jpg', 'tmp_2512.15856/./figures/top_cut-1.jpg', 'tmp_2512.15856/./figures/output_3_cut-0.png', 'tmp_2512.15856/./figures/output_3_cut-1.png', 'tmp_2512.15856/./figures/output_4_cut-0.png', 'tmp_2512.15856/./figures/output_4_cut-1.png', 'tmp_2512.15856/./figures/c_over_a_vs_M_R200.png', 'tmp_2512.15856/./figures/c_over_a_vs_M_R200_2.png', 'tmp_2512.15856/./figures/c_over_a_vs_M_R200_10.png']
copying  tmp_2512.15856/./figures/output_1_cut-0.png to _build/html/
copying  tmp_2512.15856/./figures/output_1_cut-1.png to _build/html/
copying  tmp_2512.15856/./figures/output_2_cut-0.png to _build/html/
copying  tmp_2512.15856/./figures/output_2_cut-1.png to _build/html/
copying  tmp_2512.15856/./figures/top_cut-0.jpg to _build/html/
copying  tmp_2512.15856/./f

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# The AIDA-TNG project: 3D halo shapes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15856-b31b1b.svg)](https://arxiv.org/abs/2512.15856)<mark>Appeared on: 2025-12-19</mark> -  _12 pages, and 8 figures, submitted to A&A. Find more information on AIDA-TNG here: this https URL_

</div>
<div id="authors">

C. Giocoli, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** The shapes of dark matter halos can be used to constrain the fundamental properties of dark matter. In standard Cold Dark Matter (CDM) cosmologies, halos are typically triaxial, with a preference for prolate configurations, particularly at low masses and high redshift. We focus on the characterization of total matter 3D shape in alternative dark matter models, such as Self-Interacting Dark Matter (SIDM) and Warm Dark Matter (WDM). These scenarios predict different structural properties due to collisional effects or the suppression of small-scale power. We measure the different halo component shapes -- dark matter, stars and gas -- at various radii from the center in the AIDA-TNG (Alternative Interacting Dark Matter and Astrophysics – TNG), which is a suite of high-resolution cosmological simulations built upon the IllustrisTNG framework. The intent is to systematically study how     different dark matter models -- specifically, SIDM and WDM -- affect galaxy formation and the structure of dark matter halos, when realistic baryonic physics is also included. SIDM models tend to produce rounder and more isotropic halos, especially in the inner regions, as a result of momentum exchange between dark matter particles. WDM halos are also slightly more spherical than their CDM counterparts, and are typically less concentrated. In all cases, the inclusion of self-consistent baryonic physics makes the central regions of all halos rounder, while still revealing clear distinctions among the various dark matter models. The general framework presented in this work,     based on the 3D halo shape, can be useful to interpret multi-wavelength data analyses of galaxies and clusters.

</div>

<div id="div_fig1">

<img src="tmp_2512.15856/./figures/output_1_cut-0.png" alt="Fig3.1" width="25%"/><img src="tmp_2512.15856/./figures/output_1_cut-1.png" alt="Fig3.2" width="25%"/><img src="tmp_2512.15856/./figures/output_2_cut-0.png" alt="Fig3.3" width="25%"/><img src="tmp_2512.15856/./figures/output_2_cut-1.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** Profiles of minor-to-major (top) and intermediate-to-major (bottom) axis ratios for $z=0$ halos in the CDM simulation in bins of increasing mass (from left to right).
Solid black curves show the total axis ratios in the FP run, while solid red, dotted gold, and solid orange curves show the corresponding results for the dark matter, gas, and stars, respectively. Dashed red curves show halos in the same mass bins but for the dark matter-only run. Green data points (shown only in some of the boxes) consider
the satellites and were weighted by stellar mass when computing the inertial mass tensor. The error bars bracket 25\% and 75\% of the distribution. The vertical bars display the resolution limit, which is equal to 10 times the softening length. (*fig_LCDM_Radius*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15856/./figures/top_cut-0.jpg" alt="Fig4.1" width="16%"/><img src="tmp_2512.15856/./figures/top_cut-1.jpg" alt="Fig4.2" width="16%"/><img src="tmp_2512.15856/./figures/output_3_cut-0.png" alt="Fig4.3" width="16%"/><img src="tmp_2512.15856/./figures/output_3_cut-1.png" alt="Fig4.4" width="16%"/><img src="tmp_2512.15856/./figures/output_4_cut-0.png" alt="Fig4.5" width="16%"/><img src="tmp_2512.15856/./figures/output_4_cut-1.png" alt="Fig4.6" width="16%"/>

**Figure 4. -** Minor-to-major axis ratios for the DM models relative to the CDM value, for the same ($z=0$) mass bins as Fig. \ref{fig_LCDM_Radius}. Top and bottom panels display the results from FP and DMO runs, respectively. (*reldiff_c_over_a*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15856/./figures/c_over_a_vs_M_R200.png" alt="Fig6.1" width="33%"/><img src="tmp_2512.15856/./figures/c_over_a_vs_M_R200_2.png" alt="Fig6.2" width="33%"/><img src="tmp_2512.15856/./figures/c_over_a_vs_M_R200_10.png" alt="Fig6.3" width="33%"/>

**Figure 6. -** Minor-to-major axis ratios as a function of the halo mass $M_{200}$. Red data points display the median relation for the CDM DMO simulation at $z=0$, while the corresponding
solid (dotted) error bars bracket $25$ and $75\%$ of the distribution at fixed halo mass. Black, blue, green, and magenta connected data points show the results for the FP runs in different dark matter models. To avoid overcrowding, these are displayed without the error bars.  Left, central, and right panels show the measurements at the halo boundary radius $R_{200}$, $R_{200}$/2, and $R_{200}$/10, respectively. The green dot-dashed curve shows the results from [Despali, Giocoli and Tormen (2014)](), while the red and black dotted lines show the best fit relation assuming the median value of the [Bonamigo, Despali and Limousin (2015)]() model at $\left(c/a\right)_{R200}$ -- repeated in all three panels to guide the reader. (*fig_c_over_c_M200*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15856"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# The AIDA-TNG project: dark matter profiles and concentrations in alternative dark matter models

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15869-b31b1b.svg)](https://arxiv.org/abs/2512.15869)<mark>Appeared on: 2025-12-19</mark> -  _17 pages, 12 figures, submitted to A&A. Comments welcome. Find more information on AIDA-TNG here: this https URL_

</div>
<div id="authors">

G. Despali, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** In the standard Cold Dark Matter (CDM) scenario, the density profiles of dark matter haloes are well described by analytical models linking their concentration to halo mass. Alternative scenarios, such as warm dark matter (WDM) and self-interacting dark matter (SIDM), modify the inner structure of haloes and predict different profile shapes and central slopes. We employ the AIDA-TNG simulations to investigate how alternative dark matter physics and baryonic processes jointly shape the internal structure of haloes. Using dark-matter-only and full-physics runs, we measure the dark matter density profiles of haloes spanning six orders of magnitude in mass, from $10^{9.5} M_{\odot}$ to $10^{14.5}M_{\odot}$ , and characterise them with multiple analytical models. We provide the distribution of the best-fitting parameters, as well as the concentration-mass relation in WDM and SIDM. The Einasto profile well reproduces the inner flattening produced in WDM models, both in the collisionless and in the full-physics runs. In SIDM dark-matter-only runs, haloes are better described by explicitly cored profiles, with core sizes that depend on mass and on the self-interaction model. When baryons are included, the differences between CDM and SIDM decrease, and such large dark-matter cores no longer form because adiabatic contraction in the baryon-dominated region counteracts self-interactions. Nevertheless, the coupling between baryons and self-interactions induces a broader range of inner slopes, including cases that are steeper than CDM at Milky Way masses. Alternative dark matter physics thus leaves clear signatures in the inner halo structure, even if baryons significantly reshape these differences. Our results are useful for future studies that need to predict the properties of haloes in multiple dark matter models.

</div>

<div id="div_fig1">

<img src="tmp_2512.15869/./figures/einasto_par_dark_99.png" alt="Fig5.1" width="25%"/><img src="tmp_2512.15869/./figures/yang_par_dark_99.png" alt="Fig5.2" width="25%"/><img src="tmp_2512.15869/./figures/concentration_EIN_dark_99.png" alt="Fig5.3" width="25%"/><img src="tmp_2512.15869/./figures/concentration_yang_dark_99_new.png" alt="Fig5.4" width="25%"/>

**Figure 5. -** $Top$: Mean best fit parameters of the Einasto (first row) and Y24 (second row) models, plotted as a function of halo mass at $z=0$ in the DMO runs, together with their $1\sigma$ uncertainties represented by the coloured bands. In both cases, the first two panels show the mean scale density and radius, while the third panels differ: in the case of Einasto we have the shape parameter $\alpha_{E}$(compared to the fixed range 0.18-0.2 used in previous works for CDM haloes), while in the Y24 model the core radius that we scale to the halo radius as $r_{c}/r_{200c}$(solid lines) and to the scale radius (dashed lines). In the first two panels of the second row, the dashed black lines show the scale density and radius calculated from the Einasto fit in CDM, for comparison. Instead, in the right panel, the core radius is consistent with zero for the CDM and WDM models, and it is thus only shown for self-interacting scenarios. Here, the open circles correspond to the best-fit models from Eq. \ref{eq:model}. Finally, the vertical dashed lines correspond to the value 100$M_{hm}$, calculated for WDM1 and WDM3, where it is expected that the halo concentration starts deviating from CDM. $ Bottom$: Concentration-mass relation at $z=0$ for the DMO runs. On the left (right), we show the results based on the Einasto (Y24) fit. We compare to [Ludlow, Bose and Angulo (2016)](), who also used Einasto profiles to fit CDM and WDM halo profiles, and to [Sorini, Bose and Pakmor (2025)](), who instead modelled haloes with NFW. The dotted line shows the predicted WDM concentration when applying the correction by [Lovell, Frenk and Eke (2014)]() to the measured CDM relation (black squares). In the previous section, we have demonstrated that CDM and WDM profiles are best fit by Einasto, while SIDM ones by the Y24 profiles. We highlight these with solid lines in the figure and display the remaining ones with lighter colours. (*fig:ein_mean0*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15869/./figures/profiles_dmo_mbins_67.png" alt="Fig2.1" width="33%"/><img src="tmp_2512.15869/./figures/profiles_fp_mbins_67.png" alt="Fig2.2" width="33%"/><img src="tmp_2512.15869/./figures/profiles_ratio_to_dmo_67.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** Mean dark matter profiles for haloes in 21 logarithmically-spaced bins in mass between $3\times10^{9}M_{\odot}$(orange) and $10^{14}M_{\odot}$(light blue), from the DMO (top) and FP runs (middle). From left to right, we show CDM, followed by two self-interacting models (vSIDM and SIDM1) and two WDM models (WDM3 and WDM1) . We use the 50/A run in the first four panels and the WDM1 50/B box in the rightmost one. The latter only allows us to reliably measure density profiles of $M_{200c}\geq10^{10.5}M_{\odot}$, with stronger limits on the spatial resolution. In all panels, the dashed vertical line stands for the resolution limit $3\epsilon_{DM}$. When fitting the density profile, we restrict the range further to $r\geq3\epsilon_{DM}$ and haloes with more than 1000 dark matter particles.
        Finally, the bottom panels highlights the contraction caused by baryons by showing the ratio between the dark matter profiles in the FP and DMO cases.
     (*fig:prof*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15869/./figures/res_test_dmo_67.png" alt="Fig11.1" width="50%"/><img src="tmp_2512.15869/./figures/res_test_fp_67.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Convergence tests for the dark matter profile in the 50/A boxes in the DMO (top) and FP (bottom) runs. We consider CDM, SIDM1 and vSIDM and we do not show results for the WDM models, as these behave similarly to CDM. For comparison, we also show results from the IllustrisTNG 50 Mpc run, since its resolution is higher than our best case. We plot the mean dark matter profile for four representative bins in halo mass of width 0.2 dex around the reported value. In each panel, the thick solid lines show the density profiles up to the resolution limit of each simulation, while the thinner lines help visualise where convergence breaks at lower radii.  (*fig:res1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15869"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\feii}{\ion{Fe}{ii}}$
$\newcommand{\ciii}{\ion{C}{III}]~\lambda1909\xspace}$
$\newcommand{\siiii}{\ion{Si}{III}]~\lambda1892\xspace}$
$\newcommand{\heii}{\ion{He}{II}~\lambda1640\xspace}$
$\newcommand{\nv}{\ion{N}{V}~\lambda1240\xspace}$
$\newcommand{\ovi}{\ion{O}{VI}~\lambda1034\xspace}$
$\newcommand{\mgii}{\ion{Mg}{II}~\lambda2800\xspace}$
$\newcommand{\oiii}{[\ion{O}{III}]~\lambda5007\xspace}$
$\newcommand{\oiiil}{[\ion{O}{III}]~\lambda4959\xspace}$
$\newcommand{\oiiidd}{[\ion{O}{III}]~\lambda\lambda4959,5007\xspace}$
$\newcommand{\hbeta}{H\beta~\lambda4861\xspace}$
$\newcommand{\hbetashort}{H\beta\xspace}$
$\newcommand{\hgammafull}{H\gamma~\lambda4340\xspace}$
$\newcommand{\hgamma}{H\gamma\xspace}$
$\newcommand{\civ}{\ion{C}{IV}~\lambda1549\xspace}$
$\newcommand{\lya}{Ly\alpha~\lambda1216\xspace}$
$\newcommand{\nvdd}{\ion{N}{V}~\lambda\lambda1238,1242\xspace}$
$\newcommand{\lyanv}{Ly\alpha+ \ion{N}{V}~complex\xspace}$
$\newcommand{\paa}{Pa\alpha~\lambda18751\xspace}$
$\newcommand{\pab}{Pa\beta~\lambda12818\xspace}$
$\newcommand{\pag}{Pa\gamma~\lambda10938\xspace}$
$\newcommand{\pad}{Pa\delta~\lambda10049\xspace}$
$\newcommand{\nii}{[\ion{N}{II}]~\lambda6583\xspace}$
$\newcommand{\sii}{[\ion{S}{II}]~\lambda\lambda6716,6731\xspace}$
$\newcommand{\cii}{[\ion{C}{II}]~\lambda157.6 \mum\xspace}$
$\newcommand{\siv}{\ion{S}{IV}~\lambda\lambda1394,1403\xspace}$
$\newcommand{\halpha}{H\alpha~\lambda6563\xspace}$
$\newcommand{\oiiiaur}{[\ion{O}{III}]~\lambda4363\xspace}$</div>



<div id="title">

# Shedding the envelope: JWST reveals a kiloparsec-scale [O${\sc iii}$]-weak Balmer shell around a z=7.64 quasar

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15881-b31b1b.svg)](https://arxiv.org/abs/2512.15881)<mark>Appeared on: 2025-12-19</mark> -  _Revised version submitted after the first referee review_

</div>
<div id="authors">

<mark>J. Wolf</mark>, et al. -- incl., <mark>E. Bañados</mark>, <mark>A. Dumont</mark>, <mark>S. Belladitta</mark>, <mark>S. Bosman</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Luminous quasars at the redshift frontier $z>7$ serve as stringent probes of super-massive black hole (SMBH) formation and they are thought to undergo much of their growth obscured by dense gas and dust in their host galaxies. Fully characterizing the symbiotic evolution of SMBHs and hosts requires rest-frame optical observations that span spatial scales from the broad-line region (BLR) to the interstellar and circumgalactic medium (ISM/CGM). The James Webb Space Telescope (JWST) now provides the necessary spatially resolved spectroscopy to do so. But the physical conditions that regulate the interplay between SMBHs and their hosts at the highest redshifts, especially the nature of early feedback phases, remain unclear.   We present JWST/NIRSpec Integral Field Unit (IFU) observations of J0313 $-$ 1806 at $z=7.64$ , the most distant luminous quasar known.   From the restframe optical spectrum of the unresolved quasar, we derive a black hole mass of $M_\mathrm{BH}=(1.63 \pm 0.10)\times10^9 M_\odot$ based on $\hbeta$ ( $\hbetashort$ ) and an Eddington rate of $\lambda=L/L_\mathrm{Edd}=0.80\pm 0.05$ , consistent with previous $\mgii$ -based estimates. J0313--1806 exhibits no detectable [ O III ] $\lambda\lambda4959,5007$ emission on nuclear scales (3 $\sigma$ upper limit equivalent width of $\oiii$ $<1.42  Å$ ).Most remarkably, we detect an ionized gas shell extending out to $\sim 1.8$ kpc traced by $\hbetashort$ emission that also lacks any significant [ O III ] $\lambda\lambda4959,5007$ , with a $3\sigma$ upper limit on the [ O III ] $ \lambda$ 5007 to $\hbetashort$ flux ratio of $\log_{10} \left( F(\mathrm{[OIII]})/F(\mathrm{H}\beta)\right)=-1.15$ .    Through photoionization modelling, we demonstrate that the extended emission is consistent with a thin, clumpy outflowing shell where [ O III ] is collisionally de-excited by dense gas. We interpret this structure as a fossil remnant of a recent blowout phase, providing evidence for episodic feedback cycles in one of the earliest quasars.    These findings suggest that dense ISM phases may play a crucial role in shaping the spectral properties of quasars accross cosmic time.

</div>

<div id="div_fig1">

<img src="tmp_2512.15881/./moment_maps_with_continuum-2.png" alt="Fig13" width="100%"/>

**Figure 13. -** Kinematic moment maps of $\hbeta$short around J0313-1806 and bandwidth-matched continuum. Gaussian spatial smoothing over 2.5 pixel kernel was applied (at 0$\farcs$05/pix scale). _First panel_: Flux map of $3\sigma$ detected $\hbeta$short(57 spaxels). These are highlighted by the red contour. We define this distribution as $\hbeta$short shell. Adjunct spaxels in 8-connectivity to these detection spaxels with at least S/N$_\mathrm{H\beta}> 1.5$ are also shown. The flux in the line peaks close to the quasar location, consistent with a central ionizing source. _Second panel_: Velocity offset map as traced by the the $v_{50}$ parameter. A clear velocity gradient is observed along the shell, indicating expansion or rotation of the structure. _Third panel_: Velocity dispersion, $\sigma$. Patches of large dispersions (>600 $\mathrm{km/s}$) are observed at the edges and to the south-east of the structure, potentially indicating highly turbulent gas motion. _Fourth panel_: Continuum map constructed by integrating the fitted continuum model over a line-free spectral window with bandwidth matched to that of $\hbeta$short. The absence of brightened nuclear morphology in the continuum confirms that the observed $\hbeta$short structure is not driven by background variations or continuum residuals.  (*fig:moment_maps*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15881/./spectrum_j0313.png" alt="Fig10" width="100%"/>

**Figure 10. -**  Extracted quasar spectrum within a 0$\farcs$35 aperture centred on the spaxel with the highest integrated flux. The observed spectrum, shown in black, is well fit by the \texttt{PyQSOFit} model, shown in orange. Broad $\hbeta$short and $\hgamma$ are modelled using three and two broad Gaussian components (pink lines, individual Gaussians are shown in purple) with FWHM$>1200$ km/s, and one Gaussian component with FWHM$<1200$ km/s to trace narrow components (green line).
    The $\oiii$dd emission lines are modelled with one broad and one narrow component. No narrow $\oiii$dd components are detected, while a strong contribution from Fe{\sc ii} emission (blue line) is evident \citep{park22}. The lower panel shows fit residuals. (*fig:quasar_spectrum*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15881/./ev1_fixed.png" alt="Fig2" width="100%"/>

**Figure 2. -** EV1 plane FWHM(H$\beta$) vs. $r_{\mathrm{FeII}}$. The distribution of a subset of SDSS DR16Q quasars from \citet{wu22} is shown in black (see text for selection criteria). The colourmap gives the mean $\log_{10} \mathrm{EW}([\mathrm{O III}])$ in bins of size $\Delta r_{\mathrm{FeII}} = 0.1$ and $\Delta \mathrm{FWHM}(\mathrm{H}\beta) = 500 \mathrm{km s^{-1}}$. J0313-1806 is shown as a green star. WLQs from \citet{chen24} are shown as circles (measured EW($\oiii$)) or triangles (upper limits), colour-coded according to their $\log_{10} \mathrm{EW}([\mathrm{O III}])$. J0313--1806 and several WLQs lie outside the xA regime ($r_{\mathrm{FeII}}>1$) but nonetheless show strikingly weak $\oiii$ emission compared to SDSS DR16Q quasars in the same EV1 bins. (*fig:ev1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15881"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ns}[1]{{\color{red} #1}}$
$\newcommand{\mb}[1]{{\color{brown}{\bf Maria:} #1}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\vmic}{\xi_{\rm t}}$
$\newcommand{\vmac}{V_{\rm mac}}$
$\newcommand{\EW}{W_{\lambda}}$
$\newcommand{\mA}{{\rm mÅ}}$
$\newcommand{\Elow}{E_{\rm low}}$
$\newcommand{\Eup}{E_{\rm up}}$
$\newcommand{\SH}{S\!_{\rm H}}$</div>



<div id="title">

# Observational constraints on the origin of the elements. X. Combining NLTE and machine learning for chemical diagnostics of 4 million stars in the 4MIDABLE-HR survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15888-b31b1b.svg)](https://arxiv.org/abs/2512.15888)<mark>Appeared on: 2025-12-19</mark> -  _Submitted to ApJ. 12 pages + 22 pages appendix, 7 figures_

</div>
<div id="authors">

<mark>N. Storm</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** We present 4MOST-HR resolution Non-Local Thermal Equilibrium (NLTE) Payne artificial neural network (ANN), trained on $404 793$ new FGK spectra with 16 elements computed in NLTE. This network will be part of the Stellar Abundances and atmospheric Parameters Pipeline (SAPP), which will analyse 4 million stars during the five year long 4MOST consortium 4: MIlkyway Disc And BuLgE High-Resolution (4MIDABLE-HR) survey. A fitting algorithm using this ANN is also presented that is able to fully-automatically and self-consistently derive both stellar parameters and elemental abundances. The ANN is validated by fitting 121 observed spectra of low-mass FGKM type stars, including main-sequence dwarf, subgiant and giant stars down to [ Fe/H ] $\approx -3.4$ degraded to 4MOST-HR resolution, and comparing the derived abundances with the output of the classical radiative transfer code TSFitPy. We are able to recover all 18 elemental abundances with a bias $<0.13$ and spread $<0.16$ dex, although the typical values are $<0.09$ dex for most elements. These abundances are compared to the OMEGA+ Galactic Chemical Evolution model, showcasing for the first time, the expected performance and results obtained from high-resolution spectra of the quality expected to be obtained with 4MOST. The expected Galactic trends are recovered, and we highlight the potential of using many chemical elements to constrain the formation history of the Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2512.15888/./training_distributions_6may2025_batch0-1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution in a 2D histogram of synthetic spectra used to train the Payne. Top left panel shows the Kiel diagram, while the rest are distributions as a function of [Fe/H]. All abundances are chosen uniformly random in metallicity space, except for A(O)$< 8.87$ and A(C)$< 8.7$. There are less low metallicity giant model atmospheres, resulting in slightly less spectra at low metallicities. There are also no public MARCS models above the black line in the $\teff$-$\logg$ space, resulting in a lack of computed spectra in that regime.
 (*fig:training_dist*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15888/./hr_diagram_stellar_sample.png" alt="Fig2" width="100%"/>

**Figure 2. -** Kiel diagram of the fitted stellar sample with [Fe/H] in colour with PARSEC evolutionary tracks  ([Bressan, et. al 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.427..127B))  in colour.
 (*fig:hr_dia*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15888/./systematic_errors.png" alt="Fig3" width="100%"/>

**Figure 3. -** Estimated systematic error for stellar parameters and abundances. The error was estimated by taking spread of the difference between the derived parameter from Payne and literature (for $\teff$ and $\logg$ for benchmark stars from [Heiter, et. al (2015)](https://ui.adsabs.harvard.edu/abs/2015A&A...582A..49H)) or TSFitPy.
 (*fig:systematic_error*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15888"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\hvec}[1]{\hat{\mathbf{#1}}}$
$\newcommand{\DS}{\displaystyle}$
$\newcommand{\pd}[2]{\frac{\partial #1}{\partial #2} }$
$\newcommand{\HALF}{\frac{1}{2}}$
$\newcommand{\THREEHALF}{\frac{3}{2}}$
$\newcommand{\av}[1]{\left<{#1} \right>}$
$\newcommand{\quotes}[1]{"#1"}$
$\newcommand{\tens}[1]{\mathsf{#1}}$
$\newcommand{\tJ}{\tilde{\vec{J}}}$
$\newcommand{\cE}{\mathcal{E}}$
$\newcommand{\cF}{\mathcal{F}}$
$\newcommand{\cH}{\mathcal{H}}$
$\newcommand{\cP}{\mathcal{P}}$
$\newcommand{\cQ}{\mathcal{Q}}$
$\newcommand{\cR}{\mathcal{R}}$
$\newcommand{\cS}{\mathcal{S}}$
$\newcommand{\cT}{\mathcal{T}}$
$\newcommand{\cU}{\mathcal{U}}$
$\newcommand{\cV}{\mathcal{V}}$
$\newcommand{\cW}{\mathcal{W}}$
$\newcommand{ç}{ {\boldsymbol{c}} }$
$\newcommand{\cf}{ f }$
$\newcommand{\xf}{ {\mathbf{x}_f} }$
$\newcommand{\yf}{ {\mathbf{y}_f} }$
$\newcommand{\zf}{ {\mathbf{z}_f} }$
$\newcommand{\xe}{ {\boldsymbol{x}_e} }$
$\newcommand{\ye}{ {\boldsymbol{y}_e} }$
$\newcommand{\ze}{ {\boldsymbol{z}_e} }$
$\newcommand{\RED}{\color{red}}$
$\newcommand{\BLACK}{\color{black}}$
$\newcommand{\BLUE}{\color{blue}}$
$\newcommand{\BROWN}{\color{brown}}$
$\newcommand{\MAGENTA}{\color{magenta}}$
$\newcommand{\GREEN}{\color{green}}$
$\newcommand{\mb}[1]{{\color{violet}MB: #1}}$
$\newcommand{\vec}[1]{\mathbf{#1}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# 2D or not 2D? Exploring 3D relativistic magnetic reconnection dynamics with highly accurate numerical simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.15954-b31b1b.svg)](https://arxiv.org/abs/2512.15954)<mark>Appeared on: 2025-12-19</mark> -  _Submitted to MNRAS_

</div>
<div id="authors">

V. Berta, et al. -- incl., <mark>G. Mattia</mark>

</div>
<div id="abstract">

**Abstract:** Fast reconnection in magnetically dominated plasmas is widely invoked in models of dissipation in pulsar winds, gamma-ray flares in the Crab nebula, and to explain the radio nanoshots of pulsars.When current sheets evolve reaching a critical inverse aspect ratio, scaling as $S^{-1/3}$ with the plasma Lundquist number, the so-called _ideal_ tearing instability sets in, with modes growing, independently of $S$ , extremely rapidly on timescales of only a few light-crossing times of the sheet length.We present the first set of fully 3D simulations of current-sheet disruption triggered by the ideal tearing instability within the resistive relativistic MHD approximation, as appropriate in situations where the Alfvén velocity approaches the speed of light.We compare 3D setups with different initial conditions with their 2D counterparts, and we assess the impact of dimensionality and of the magnetic field topology on the onset, evolution, and efficiency of reconnection.In force-free configurations, 3D runs develop ideal tearing, secondary instabilities, and a thick, turbulent current layer, sustaining dissipation of magnetic energy longer than in 2D.In pressure-balanced current sheets with a null guide field, 2D reference runs show the familiar reconnection dynamics, whereas in 3D tearing dynamics is quenched after the linear phase, as pressure-driven modes growing on forming plasmoids outcompete plasmoid coalescence and suppress fast dissipation of magnetic energy.Taken together, these results suggest that the evolution and efficiency of reconnection depend sensitively on the local plasma conditions and current-sheet configuration, and can be properly captured only in fully 3D simulations.

</div>

<div id="div_fig1">

<img src="tmp_2512.15954/./Figures/PB/PB_SUS_2D_vs_3D.png" alt="Fig3" width="100%"/>

**Figure 3. -** \small Temporal evolution of the root-mean-square of the magnetic field component $B_{x}$(upper left panel), the module of $|\nabla \times $\vec${B}|$ as proxy for the non-relativistic electric current $|J|$(upper right), electromagnetic energy normalized to its initial value ${\cal E}_{\text{EM}} =(E^2 + B^2)/(E_0^2 + B_0^2)$(lower left), and the average relativistic temperature ${\Theta} = p/\rho$(lower right) for simulations initialized with the pressure-balanced equilibrium. All quantities integrated over the uniform portion of the computational volume are indicated with the $\av $ symbol.
  Time is given in units of the light-crossing time of half the sheet length, $\tau_c = L/c$.
  The black dotted and dash-dotted vertical lines mark, respectively, the transition to the non-linear regime. In the upper-left panel, the green dash-dotted line identifies the time at which the linear phase terminates in the corresponding force-free reference run, and the red dashed line marks the slope of the linear growth rate of ideal tearing in models \texttt{ff2d} and \texttt{ff3d}.
 (*fig:PB_time_evolution*)

</div>
<div id="div_fig2">

<img src="tmp_2512.15954/./Figures/FF/FF_SUS_2D_vs_3D.png" alt="Fig1" width="100%"/>

**Figure 1. -** \small Temporal evolution of the root-mean-square of the magnetic field component $B_{x}$(upper left panel), the magnitude of the total electric current $|J|$(upper right), the electromagnetic energy ${\cal E}_{\text{EM}} = (E^2 + B^2)/(E_0^2 + B_0^2)$ normalized to its initial value (bottom-left), and the temperature ${\Theta} = p/\rho$(bottom-right) for simulations initialized with the force-free equilibrium.
  All volume-integrated quantities are indicated with the $\av $ symbol.
  Time is given in units of the light-crossing time of half the sheet length, $\tau_c = L/c$.
  The black dotted and dash-dotted vertical lines mark, respectively, the transition to the non-linear regime.
 (*fig:FF_time_evolution*)

</div>
<div id="div_fig3">

<img src="tmp_2512.15954/./Figures/FF/FF_z0_cut.png" alt="Fig2" width="100%"/>

**Figure 2. -** \small Snapshot of the module of the $z$-component of $|\nabla \times $\vec${B}|$ used as a proxy for $|J_z|$ in the plane $z = 0$ for model \texttt{ff2d}(left column), and module \texttt{ff3d}(right column) at three different times: $t = 5\tau_c$(top row), $t = 10\tau_c$(middle row), $t = 15\tau_c$(bottom row).
 (*fig:FF_xy_plane*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.15954"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\strike}[1]{\sout{#1}}$
$\newcommand{\knedit}[1]{\textcolor{purple}{#1}}$
$\newcommand{\lsedit}[1]{\textcolor{brown}{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# How massive neutrinos reshape the cosmic web

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.16517-b31b1b.svg)](https://arxiv.org/abs/2512.16517)<mark>Appeared on: 2025-12-19</mark> -  _10 pages, 6 figures_

</div>
<div id="authors">

L. N. L. Simões, et al. -- incl., <mark>K. Naidoo</mark>

</div>
<div id="abstract">

**Abstract:** We explore the effects of massive neutrinos on the cosmic web using the FLAMINGO simulations. We classify the cosmic web into voids, sheets, filaments, and clusters, and find that massive neutrinos affect the environment by decreasing the volume occupied by clusters and voids. We find that increasing the neutrino mass shifts the volume-weighted density distribution towards higher densities and leads to a more narrow density distribution, which we interpret as neutrinos delaying structure formation. We construct the minimum spanning tree (MST) graph from the subhaloes, adopting a number density chosen to match that expected for DESI-like observations. We show that most MST edges lie in filaments, approximately $70\%$ throughout different simulations, which we link to its sensitivity to neutrino mass. We also link the MST's edge length signal at different scales to different cosmic web environments, with clusters dominating the signal at small scales, voids at longer scales, and filaments at intermediate scales. The strong correlation between MST edges and cosmic web environments reinforces the MST’s potential to be used as a classifier for large-scale structure in galaxy surveys. We compare the effects of baryonic physics and massive neutrinos and find that each produces distinct signatures in MST edge lengths. This analysis is performed in 3D space, using the true positions of subhaloes and not accounting for redshift space distortions. Nevertheless, these results emphasise the MST's capability to go beyond two-point statistics, motivating future applications to real observational data.

</div>

<div id="div_fig1">

<img src="tmp_2512.16517/./images/NoNeut_edges_rebinned.png" alt="Fig5.1" width="50%"/><img src="tmp_2512.16517/./images/mst_PDF_rebinned.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Histograms of the distribution of the edge lengths ($l$) in the MST separated into each classification of the cosmic web that they are tracing. In Fig. \ref{fig:neut_edges} we plot the total probability distribution function (PDF), normalised to total counts. In Fig. \ref{fig:neut_PDF_edges} we plot the individual PDFs, normalised for each classification. These plots come from the analysis of the DMO$+\nu$ simulation with $M_\nu = 0.06 \:{\rm eV }$, but the relations are similar for all simulations. (*fig:edges_neutrinos*)

</div>
<div id="div_fig2">

<img src="tmp_2512.16517/./images/combined_density_mass_fraction.png" alt="Fig2" width="100%"/>

**Figure 2. -** The normalised density (top) and mass-weighted density (middle) distributions for each neutrino mass. The top panels show the total density whereas in the bottom panels we subtract the $M_{\nu}=0.06\:{\rm eV  }$ distribution from the other two. In the bottom panel we split by cosmic web environment to show which components dominate at different densities. (*fig:density_models*)

</div>
<div id="div_fig3">

<img src="tmp_2512.16517/./images/MSTripleStats_HYDROonly.png" alt="Fig6.1" width="50%"/><img src="tmp_2512.16517/./images/MSTripleStats_DMOvsHydro.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** A comparison between the histogram distributions of the MST statistics in different models, using 500 000  haloes for the MST. In Fig. \ref{fig:neut_MST} we compare different neutrinos masses, and in Fig. \ref{fig:Baryon_MST} we compare the effect of baryons and the effect of massive neutrinos. In the top panels we show the distributions of the degrees (_d_), edge lengths (_l_), branch lengths (_b_), and branch shapes (_s_). The bottom panels show the differences between the PDFs, subtracting the initial $M_\nu = 0.06\:{\rm eV  }$ distribution (the dashed black line shows zero) from each model. (*fig:mst_comp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.16517"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

135  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

10  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
